# Datamodels Retriever Experiment

This document has the goal to show the implementation of a context retriever using Datamodels and its comparison against classical approaches

# 1. Run Classical Approaches

First it's needed to run the comparison subject, to do so we will use the script present in this folder, it's just necessary to run

```
python run_classical_retriever.py
```

This will run the retriever for each sample from the test dataset, saving the data at every 50 samples as checkpoint

# 2. Split Data for Datamodeling

Here we will be spliting the data to achieve a dev dataset containing a representative numbe of samples to each subtask
The "k" used here is 15

In [1]:
from src.utils import split_dev_set, subset_df
from src.retriever import DatamodelsRetriever
from src.datamodels_pipeline import DatamodelPipeline, DatamodelConfig, MemMapConfig
from src.evaluator import Rouge_L_evaluator
from src.llms import Llama3_1
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import os

# Limit available GPUs to GPU 0 and 1
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


/home/caio.rhoden/miniconda3/envs/datamodels/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# train = pd.read_csv("../../data/instruction-induction-data/processed/induce_tasks_examples.csv")
# train_subset = subset_df(train, 200, "task")
# train_subset.to_csv("../../data/instruction-induction-data/processed/train.csv")

# split_dev_set(
#     path="../../data/instruction-induction-data/processed/train.csv",
#     saving_path="../../data/instruction-induction-data/datamodels",    
#     k_samples=15,
#     task_column="task",
# )

## 3. Split the Collections to be trained

In [3]:
#### First time create collection #####
# retriever = DatamodelsRetriever(k=8)
# retriever.create_collections_index(
#     "../../data/instruction-induction-data/datamodels_15_10_2024/train_set.csv",
#     "../../data/instruction-induction-data/datamodels_15_10_2024",
#     n_samples=500,
#     test_per=0.2,

# )

In [4]:
llama = Llama3_1()
llama.run("What is the best vegatable for salad")

Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.07s/it]


'What is the best vegatable for salad?\nI am looking for a vegetable that I can use in a salad.'

In [5]:
######### Datamodels config for experiment #########
config = DatamodelConfig(
    k = 8,
    num_models= 315,
    datamodels_path = "../../data/instruction-induction-data/datamodels_15_10_2024",
    train_collections_idx = None,
    test_collections_idx = None,
    test_set = None,
    train_set = None,
    instructions= None,
    llm = llama,
    evaluator=Rouge_L_evaluator(),
)

In [6]:
datamodel = DatamodelPipeline(config)
datamodel.set_collections_index()
datamodel.set_dataframes()
datamodel.set_instructions_from_path()

Loaded train collection index
Loaded train set
Loaded test set


In [7]:
# datamodel._load_collections_from_path()

In [8]:
import torch
train_dataset = torch.load(f"{datamodel.datamodels_path}/datasets/train_dataset.pt")

/tmp/ipykernel_3972709/3035896702.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_dataset = torch.load(f"{datamodel.datamodels_path}/datasets/train_dataset.pt")


In [9]:
print("Len Train Dataset: ", len(train_dataset))
print("Len Train Mask Shape", train_dataset[0][0].shape)
print("Len Train Target Shape", train_dataset[0][1].shape)

Len Train Dataset:  315
Len Train Mask Shape torch.Size([401, 3885])
Len Train Target Shape torch.Size([401])


In [10]:
datamodel.train_datamodels(
    epochs=40,
    batch_size=100,
    val_split=0.1,
    lr=0.001,
    random_seed=42,
    device="cuda:0",
)

/work/caio.rhoden/datamodels-context-reduction/src/datamodels_pipeline/Datamodels.py:197: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_dataset = torch.load(f"{self.da

Idx: 0
Epoch [1/40], Loss: 8.4561, Val MSE: 8.5924
Epoch [2/40], Loss: 8.1865, Val MSE: 8.3050
Epoch [3/40], Loss: 7.9254, Val MSE: 8.0271
Epoch [4/40], Loss: 7.6726, Val MSE: 7.7584
Epoch [5/40], Loss: 7.4279, Val MSE: 7.4986
Epoch [6/40], Loss: 7.1911, Val MSE: 7.2475
Epoch [7/40], Loss: 6.9618, Val MSE: 7.0048
Epoch [8/40], Loss: 6.7398, Val MSE: 6.7701
Epoch [9/40], Loss: 6.5249, Val MSE: 6.5433
Epoch [10/40], Loss: 6.3168, Val MSE: 6.3240
Epoch [11/40], Loss: 6.1154, Val MSE: 6.1120
Epoch [12/40], Loss: 5.9204, Val MSE: 5.9071
Epoch [13/40], Loss: 5.7317, Val MSE: 5.7091
Epoch [14/40], Loss: 5.5489, Val MSE: 5.5176
Epoch [15/40], Loss: 5.3720, Val MSE: 5.3325
Epoch [16/40], Loss: 5.2008, Val MSE: 5.1537
Epoch [17/40], Loss: 5.0350, Val MSE: 4.9807
Epoch [18/40], Loss: 4.8745, Val MSE: 4.8136
Epoch [19/40], Loss: 4.7191, Val MSE: 4.6521
Epoch [20/40], Loss: 4.5686, Val MSE: 4.4959
Epoch [21/40], Loss: 4.4230, Val MSE: 4.3450
Epoch [22/40], Loss: 4.2820, Val MSE: 4.1992
Epoch [23/40

In [17]:
# datamodel.create_pre_collection(start_idx=0, end_idx=1)

# template = """
#             Fill the expected Output according to the instruction
#             Intruction: {instruction}

#             Examples:
#             {context}

#             User Input:
#             {input}

#             Model Output:
#         """

Collection id: 0
Checkpoint 0 saved


In [3]:
df = pd.read_pickle( "../../data/instruction-induction-data/datamodels_15_10_2024/pre_collections/15-10-2024/pre_collection_0.pickle")
df.head()

,collection_idx,test_idx,input,predicted_output,true_output,optinal_output
0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",\n Fill the expected Output accordi...,The manager was mentioned by the judge.,NaN
1,0,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",\n Fill the expected Output accordi...,The presidents were encouraged by the professors.,NaN
2,0,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",\n Fill the expected Output accordi...,The secretary was recommended by the banker.,NaN
3,0,3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",\n Fill the expected Output accordi...,The presidents were thanked by the secretaries.,NaN
4,0,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",\n Fill the expected Output accordi...,The doctor was recognized by the bankers.,NaN


In [4]:




def extract_output(text):
    # Split the string and strip leading/trailing spaces and newlines
    return text.split(" Model Output:\n ", 1)[-1].strip()

df = pd.read_pickle( "../../data/instruction-induction-data/datamodels_15_10_2024/pre_collections/15-10-2024/pre_collection_0.pickle")


result = Rouge_L_evaluator().evaluate(df["true_output"].to_numpy(),  df["predicted_output"].apply(extract_output).to_numpy(), datamodel.llm.tokenizer)


In [7]:
result

array([1.        , 0.71428571, 1.        , 1.        , 1.        ,
       0.77777778, 1.        , 1.        , 0.77777778, 1.        ,
       1.        , 0.77777778, 1.        , 1.        , 1.        ,
       0.28571429, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.66666667, 1.        , 0.        ,
       0.        , 0.        , 0.        , 0.4       , 0.        ,
       0.33333333, 0.        , 1.        , 0.28571429, 0.        ,
       0.5       , 0.        , 0.        , 0.        , 0.5       ,
       0.        , 0.28571429, 0.        , 1.        , 0.        ,
       1.        , 0.        , 0.        , 0.        , 0.4       ,
       0.        , 0.        , 0.18181818, 0.        , 1.        ,
       0.        , 1.        , 1.        , 1.        , 0.        ,
       0.        , 0.        , 0.        , 0.66666667, 0.28571429,
       0.        , 0.4       , 1.        , 0.33333333, 0.        ,
       0.33333333, 0.        , 0.66666667, 0.28571429, 0.33333

In [8]:
df_collection = pd.read_pickle( "../../data/instruction-induction-data/datamodels_15_10_2024/collections/15-10-2024/pre_collection_10.pickle")
df_collection.head()


,collection_idx,test_idx,input,evaluation
0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.000000
1,0,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.714286
2,0,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.000000
3,0,3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.823529
4,0,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.777778


## Result Analysis

In [3]:
traind_df = pd.read_csv("../../data/instruction-induction-data/datamodels_15_10_2024/train_set.csv")
traind_df.head()

,task,input,output,possible_outputs
0,active_to_passive,The lawyer avoided the secretaries.,The secretaries were avoided by the lawyer.,NaN
1,active_to_passive,The judges supported the scientists.,The scientists were supported by the judges.,NaN
2,active_to_passive,The secretary recommended the professor.,The professor was recommended by the secretary.,NaN
3,active_to_passive,The artists recommended the secretary.,The secretary was recommended by the artists.,NaN
4,active_to_passive,The secretary avoided the senator.,The senator was avoided by the secretary.,NaN


In [4]:
dev_df = pd.read_csv("../../data/instruction-induction-data/datamodels_15_10_2024/dev_set.csv")
dev_df.head()

,task,input,output,possible_outputs
0,active_to_passive,The judge mentioned the manager.,The manager was mentioned by the judge.,NaN
1,active_to_passive,The professors encouraged the presidents.,The presidents were encouraged by the professors.,NaN
2,active_to_passive,The banker recommended the secretary.,The secretary was recommended by the banker.,NaN
3,active_to_passive,The secretaries thanked the presidents.,The presidents were thanked by the secretaries.,NaN
4,active_to_passive,The bankers recognized the doctor.,The doctor was recognized by the bankers.,NaN


In [5]:
pre_collection = pd.read_pickle( "../../data/instruction-induction-data/datamodels_15_10_2024/pre_collections/15-10-2024/pre_collection_10.pickle")
pre_collection.head()

,collection_idx,test_idx,input,predicted_output,true_output,optinal_output
0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",\n Fill the expected Output accordi...,The manager was mentioned by the judge.,NaN
1,0,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",\n Fill the expected Output accordi...,The presidents were encouraged by the professors.,NaN
2,0,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",\n Fill the expected Output accordi...,The secretary was recommended by the banker.,NaN
3,0,3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",\n Fill the expected Output accordi...,The presidents were thanked by the secretaries.,NaN
4,0,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",\n Fill the expected Output accordi...,The doctor was recognized by the bankers.,NaN


In [6]:
collection = pd.read_pickle( "../../data/instruction-induction-data/datamodels_15_10_2024/collections/15-10-2024/pre_collection_10.pickle")
collection.head()

,collection_idx,test_idx,input,evaluation
0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.000000
1,0,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.714286
2,0,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.000000
3,0,3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.823529
4,0,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.777778


## Single Sample Demonstration

In [7]:
### Anlyse a sing sample
sample = pre_collection.loc[0]
sample["evaluation"] = collection.copy().loc[0]["evaluation"]
sample["task"] = dev_df.loc[sample["test_idx"]]["task"]


/tmp/ipykernel_59303/1521282305.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample["evaluation"] = collection.copy().loc[0]["evaluation"]
/tmp/ipykernel_59303/1521282305.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample["evaluation"] = collection.copy().loc[0]["evaluation"]
/tmp/ipykernel_59303/1521282305.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample["task"] = dev_df.loc[sample["test_idx"]]["task"]
/tmp/ip

In [8]:
print(sample["input"])

[0 0 0 ... 0 0 0]


In [9]:
print(sample["predicted_output"])


            Fill the expected Output according to the instruction
            Intruction: Write the input sentence in passive form.

            Examples:
            Input: 5040 
Output: five thousand and forty
Input: 1 64 
Output: 65
Input: grenade 
Output: r
Input: Sentence 1: A school bus is driving uphill on a rural road. Sentence 2: A race care driving along a dirt road. 
Output: 1 - probably not
Input: souvenir 
Output: near
Input: The student recognized the professors. 
Output: The professors were recognized by the student.
Input: Sentence 1: White House in damage control over Obama Supreme Court remarks Sentence 2: Fact check: Obama's Supreme Court remarks 
Output: 4 - almost perfectly
Input: camouflage 
Output: c


            User Input:
            The judge mentioned the manager.

            Model Output:
         The manager was mentioned by the judge.




In [10]:
print(sample["true_output"])

The manager was mentioned by the judge.


In [11]:
print(sample["evaluation"])

1.0


## Evaluation Results by Task

In [12]:
samples = pre_collection.copy()
samples["evaluation"] = collection["evaluation"]
samples["task"] = samples["test_idx"].apply(lambda idx: dev_df.loc[idx, 'task'])

In [13]:
samples["task"].value_counts()

task
active_to_passive          165
antonyms                   165
diff                       165
first_word_letter          165
larger_animal              165
letters_list               165
negation                   165
num_to_verbal              165
orthography_starts_with    165
rhymes                     165
second_word_letter         165
sentence_similarity        165
sentiment                  165
singular_to_plural         165
sum                        165
synonyms                   165
taxonomy_animal            165
translation_en-de          165
translation_en-es          165
translation_en-fr          165
word_in_context            165
Name: count, dtype: int64

In [15]:
task_results = samples.groupby('task').agg(
    mean=('evaluation', 'mean'),
    q25=('evaluation', lambda x: x.quantile(0.25)),
    q50=('evaluation', lambda x: x.quantile(0.50)),  # Same as median
    q75=('evaluation', lambda x: x.quantile(0.75)),
    q100=('evaluation', lambda x: x.quantile(1.00))
).reset_index()

len(task_results)

21

In [16]:
task_results

,task,mean,q25,q50,q75,q100
0,active_to_passive,0.498462,0.000000,0.500000,0.823529,1.00
1,antonyms,0.074574,0.000000,0.000000,0.000000,1.00
2,diff,0.331053,0.000000,0.285714,0.400000,1.00
3,first_word_letter,0.136107,0.000000,0.000000,0.166667,1.00
4,larger_animal,0.349500,0.000000,0.285714,0.666667,1.00
5,letters_list,0.241986,0.000000,0.000000,0.571429,1.00
6,negation,0.521000,0.266667,0.588235,0.800000,1.00
7,num_to_verbal,0.428086,0.000000,0.500000,0.800000,1.00
8,orthography_starts_with,0.164283,0.000000,0.181818,0.222222,1.00
9,rhymes,0.254800,0.000000,0.000000,0.400000,1.00
